In [1]:
import pandas as pd 
import wandb
from multiprocessing import Pool
from os import cpu_count
from tqdm.auto import tqdm
import time
from pathlib import Path


In [2]:
path = Path("./raw_data/")  

In [68]:
meta = []
for p in tqdm(list(path.iterdir())):
    if "_" not in p.name:
        continue
    meta.append(
        {
            "id": p.name.split("_")[1].removesuffix(".tsv"),
            "min_steps": pd.read_csv(p, nrows=5, sep="\t")["step"].min(),
            "filename": str(p),
        }
    )



  0%|          | 0/140 [00:00<?, ?it/s]

In [69]:
runs_df = pd.read_csv(path / "runs.tsv", sep="\t")

In [70]:
runs_df = pd.merge(runs_df, pd.DataFrame(meta), on="id", how="left")

In [71]:
order = ["group", "name", "id", "model_size", "seed", "min_steps", "max_steps"]
runs_df = (
    runs_df[order]
    .sort_values(["model_size", "seed", "min_steps", "max_steps"])
    .assign(total_steps=lambda _df: _df["max_steps"] - _df["min_steps"] + 1)
    .assign(selected=lambda _df: _df["total_steps"] == 143_000)
)

In [72]:
runs_df.to_csv("./data/runs.tsv", index=False, sep="\t")

In [73]:
check_df = (
    runs_df
    .groupby(["model_size", "seed"])["selected"].unique().map(max)
    .reset_index()
    .query("selected != True")
)
check_df

,model_size,seed,selected
9,160m,4,False
14,160m,9,False
15,410m,1,False
16,410m,2,False
17,410m,3,False
18,410m,4,False
19,410m,5,False
20,410m,6,False
21,410m,7,False
22,410m,8,False


In [77]:
r = pd.merge(runs_df, check_df.drop(columns=["selected"]), on=["model_size", "seed"], how="inner")

In [89]:
r["interval"] = r["min_steps"].astype(str) + "-" + r["max_steps"].astype(str)

In [97]:
pd.set_option('display.max_colwidth', 120)

In [98]:
r.groupby(["model_size", "seed"]).agg(
    total_steps=("total_steps", "sum"),
    intervals=("interval", "unique"),
)

total_steps  \
model_size seed                
160m       4          143335   
           9          144278   
410m       1          143000   
           2          143000   
           3          143000   
           4          143890   
           5          147075   
           6          145204   
           7           77662   
           8          145667   
           9          144573   
70m        5          287256   
           6          286624   
           7          289842   
           8          287341   

                                                                                                                               intervals  
model_size seed                                                                                                                           
160m       4                                                                                        [1-61185, 61001-61150, 61001-143000]  
           9                                                                           [1-22623, 22001-22008, 22001-31647, 31001-143000]  
410m       1                   [1-10000, 10001-29000, 29001-30000, 30001-32000, 32001-41000, 41001-105000, 105001-116000, 116001-143000]  
           2                                                     [1-3000, 3001-4000, 4001-29000, 29001-50000, 50001-88000, 88001-143000]  
           3     [1-7000, 7001-15000, 15001-16000, 16001-17000, 17001-34000, 34001-43000, 43001-60000, 60001-73000, 73001-75000, 7500...  
           4                              [1-7000, 7001-55841, 55001-104000, 104001-121000, 121001-122000, 122001-122049, 122001-143000]  
           5                 [1-81715, 81001-83463, 83001-95416, 95001-95427, 95001-124741, 124001-127586, 127001-137727, 137001-143000]  
           6                                                            [1-61003, 61001-63474, 63001-75835, 75001-104892, 104001-143000]  
           7                                                 [1-399, 1-18000, 257-381, 257-44907, 44001-53341, 53001-53038, 53001-58108]  
           8     [1-666, 513-1666, 1001-2252, 2001-23026, 23001-29392, 29001-29376, 29001-39344, 39001-74999, 75001-75078, 75001-9038...  
           9                                        [1-2551, 2001-6024, 6001-29139, 29001-39999, 40001-73047, 73001-92813, 92001-143000]  
70m        5                                                                          [1-114498, 1-128758, 114001-143000, 128001-143000]  
           6                                                          [1-3, 1-129588, 5-93015, 93001-95022, 95001-143000, 129001-143000]  
           7     [1-15, 1-255, 17-105999, 257-16437, 16001-143000, 106001-107963, 107001-108637, 108001-108120, 108001-108184, 108001...  
           8                                             [1-15, 1-11538, 17-43493, 11001-143000, 43001-43011, 43001-43300, 43001-143000]